In [1]:
%%capture
import random, sys, csv, datetime, re, pandas as pd, numpy as np, math
from pandas import DataFrame, Series, date_range, concat
import graphlab as gl;
import TastingNoteParser as tnp
import GoldDataGenerator as gdg
import GoldDataFunctions as gdf

In [2]:
def guessWine(tastingNote):
    grid=tnp.getAttributes(tastingNote)
    tasting_note = gl.SFrame(data=pd.DataFrame([grid]))
    print
    print 'Guesses'
    print model.predict_topk(tasting_note,k=5, output_type='rank')
    print 'Supporting Features'
    model.get_feature_importance().to_dataframe().to_csv('feature_importance_test.csv')
    print rankFeatures(tasting_note, model.get_feature_importance())
    return (tasting_note, model.get_feature_importance())
def rankFeatures(note,features):
    nt=note.to_dataframe().transpose()
    f=features.to_dataframe()
    ntr=nt.reset_index(level=0)
    ntr.columns = ['name','index']
    a=ntr.apply(lambda x: [x[0],True] if x[1]==1 else [x[0],0] if x[1]==0 else x, axis=1)
    a.columns=['feature','value']
    f.columns=['feature','value','count']
    return pd.merge(f,a, on=['feature','value'],suffixes=('_x', '_y')).sort_values(by='count',ascending=False)

In [3]:
model=gl.load_model('boosted_trees_wine_classifier')

Use this tasting note builder:
https://docs.google.com/spreadsheets/d/1f7uo4xm2tq4oQr0x4rsf0UAv0z9GNKloAUBJSPYjzNQ/edit#gid=902392245

In [18]:
tn = 'yellow, moderate color_concentration, dry, moderate alcohol, high acid, , , , , pommaceous_fruit, , low_terpenes, , , oxidation, , , '			

In [19]:
note, features = guessWine(tn);

sweetness recognized as dry
hue recognized as yellow
color_concentration recognized as moderate
alcohol recognized as moderate
acid recognized as high
recognized low_terpenes
recognized pommaceous_fruit
recognized oxidation

Guesses
+----+----------------------+------+
| id |        class         | rank |
+----+----------------------+------+
| 0  |     Savennieres      |  0   |
| 0  | Riesling (Australia) |  1   |
| 0  |       Albarino       |  2   |
| 0  |  Riesling (Alsace)   |  3   |
| 0  |  Alsace Pinot Gris   |  4   |
+----+----------------------+------+
[5 rows x 3 columns]

Supporting Features
               feature     value  count
0                  hue    yellow    300
1              alcohol  moderate    291
2                 acid      high    282
3       residual_sugar       dry    260
4  color_concentration  moderate    240


In [15]:
features[features['name']=='high_terpenes']

name,index,count
high_terpenes,None,345


In [12]:
rankFeatures(note,features)

,feature,value,count
0,hue,straw,320
1,alcohol,moderate,291
2,acid,high,282
3,residual_sugar,dry,260
4,color_concentration,moderate,240


In [8]:
model.show(view='Tree',tree_id=0, vlabel_hover=True)

Canvas is accessible via web browser at the URL: http://localhost:58140/index.html
Opening Canvas in default web browser.
